## Implementation of Variable Elimination to approximate probability queries from a Bayesian Network.

In [11]:
# Given an arbitrary Bayesian Network, an arbitrary set of query variables, and arbitrary evidence, we want to be able to calculate the probability of said variable...
import numpy as np

def join(first: np.array, second: np.array, row_pairings: dict[int,list[int]]) -> np.array:
    """Given two distributions and the variable to to join on, join the two distributions into a third

    Args:
        first (np.ndarray): first distribution (smaller or same size)
        second (np.ndarray): second distribution (larger or same size)
        row_pairings (dict[int,list[int]]): map of rows of smaller first distribution mapping to rows of second distribution they will multiply

    Returns:
        np.ndarray: resulting distribution after joining
    """
    result = np.zeros(second.shape)
    for first_index, second_indices in row_pairings.items():
        for s in second_indices:
            result[s] += first[first_index] * second[s]
    return result

def eliminate(distribution: np.array, pairings: list[tuple[int,int]]) -> np.array:
    """eliminate a variable by summing over the distribution at the specified index pairs

    Args:
        distribution (np.ndarray): pairs of indices we add together

    Returns:
        np.ndarray: resulting distribution (dimensions will be half as large)
    """
    result = np.array(shape=len(pairings))
    for i, pair in enumerate(pairings):
        result[i] = distribution[pair[0]] + distribution[pair[1]]
    return result

In [4]:
distr_1 = np.array([.000999, .00029, .93906, .00095])
distr_2 = np.array([.95,.05,.55,.45,.17,.83,.23,.77])
joined = join(distr_1, distr_2, {0:[0,1],1:[2,3],2:[4,5],3:[6,7]})

In [12]:
eliminate(joined, pairings=[(0,2),(1,3),(4,6),(5,7)])

array([1.108550e-03, 1.804500e-04, 1.598587e-01, 7.801513e-01])

In [10]:
import numpy as np

def find_row_sum_pairs(var: int, all_vars: list[int]) -> list[tuple[int,int]]:
    """Given a variable that should be eliminated by summing over its possibilities, and the ordered list of all variables, find all of the pairs of rows that should be summed together to eliminate said variable

    Args:
        var (int): variable to eliminate
        all_vars (list[int]): ordered list of all variables

    Returns:
        list[tuple[int,int]]: list of pairs of rows that should be summed together in the corresponding array of values
    """
    idx = all_vars.index(var)
    binary_posn = len(all_vars) - 1 - idx
    row_pairs = []
    for i in range(2**(len(all_vars)-1)):
        # this is how many row pairs there will be
        first = (2**binary_posn)*(2*i)
        second = first + 2**binary_posn
        row_pairs.append([first,second])
    return row_pairs

def find_corresponding_rows(bit_mask: list[int], common_vars: list[int], all_vars: list[int]) -> list[int]:
    """Given a list of common variables and which ones are set to true, find the rows in the distribution array that would correspond to all_vars which share the same truth value

    Args:
        bit_mask (list[int]): truth values for each of the common variables
        common_vars (list[int]): list of common variables
        all_vars (list[int]): list of all variables (common variables will be a subset)

    Returns:
        list[int]: list of rows corresponding with the truth values associated with the common variables
    """
    vars_to_posn = {v : all_vars.index(v) for v in common_vars}
    common_rows = set()
    # we will have a list of sets which we will take the intersection of
    row_sets = [set() for _ in common_vars]
    for i, v in enumerate(common_vars):
        idx = vars_to_posn[v]
        binary_posn = len(all_vars) - 1 - idx
        switch_every = 2 ** binary_posn
        # see if at this row, the given common variable's truth value is matched
        for row in range(2**len(all_vars)):
            negative_row = ((row // switch_every) % 2) == 0
            if not bit_mask[i] and negative_row:
                row_sets[i].add(row)
            elif bit_mask[i] and not negative_row:
                row_sets[i].add(row)
    # now take the intersection of all the row sets
    for row in row_sets[0]:
        missing = False
        for i in range(1, len(row_sets)):
            if row not in row_sets[i]:
                missing = True
                break
        if not missing:
            common_rows.add(row)

    result = list(common_rows)
    result.sort()
    return result

def find_common_rows(prev_factor_vars: list[int], next_factor_vars: list[int]) -> tuple[dict[int,list[int]],list[int]]:
    """Given two (ordered) lists of variables, determine which rows of the second factor must be multiplied by each row of the first factor

    Args:
        prev_factor_vars (list[int]): variables corresponding to first factor/distribution
        next_factor_vars (list[int]): variables corresponding to second factor/distribution

    Returns:
        tuple[dict[int,list[int]],list[int]]: for each row of the first factor, which rows of the second factor should it multiply? Also, return the variable order for the joined array.
    """
    common_vars = [v for v in prev_factor_vars if v in next_factor_vars]
    common_vars_set = set(common_vars)
    
    # Each row in the previous factor corresponding to one combination of the values for the variables in common must multiply each row in the second factor 
    row_multiplications = {}
    # Calculate multiplication consistencies
    for i in range(2**len(common_vars)):
        # i represents the bit mask for all the binary values of the common variables
        # for each appearance of these shared variables, all such rows in the first factor multiply by all such rows in the second factor
        bit_mask = [0 for _ in range(len(common_vars))]
        for j in range(len(common_vars)):
            # see which variables are set to true by creating the bit mask
            if ((1 << j) & i) > 0:
                bit_mask[j] += 1
        # now we have a bit mask telling us which variables will be true - find the rows for each of the two factors where this combination of truth values for the shared variables occurs
        prev_factor_rows = find_corresponding_rows(bit_mask, common_vars, prev_factor_vars)
        next_factor_rows = find_corresponding_rows(bit_mask, common_vars, next_factor_vars)
        for prev_row in prev_factor_rows:
            row_multiplications[prev_row] = []
            for next_row in next_factor_rows:
                row_multiplications[prev_row].append(next_row)

    joined_vars = [v for v in prev_factor_vars] + [v for v in next_factor_vars if v not in common_vars_set]
    return (row_multiplications, joined_vars)

def eliminate_variable(var: int, relevant_factors: list[tuple[list[int],np.array]]) -> tuple[list[int],np.array]:
    """Given which variable we want to eliminate, multiply its respective factors together and then sum out over the variable to eliminate

    Args:
        var (int): the variable to eliminate
        factors (list[tuple[list[int],np.array]]): list of distributions corresponding to this variable

    Returns:
        tuple[list[int],np.array]: new list of relevant variables to this array along with the array (a new distribution)
    """
    # We need to find out which rows multiply together
    # Break into pairs
    prev_factor = relevant_factors[0]
    for i in range(1, len(relevant_factors)):
        next_factor = relevant_factors[i]
        # Find the list of rows corresponding to each factor that must multiply together
        row_multiplications, joined_vars = find_common_rows(prev_factor[0], next_factor[0])
        # Create a new array to populate with products
        merged_array = np.zeros(shape=2**len(joined_vars))
        current_idx = 0
        for prev_idx in row_multiplications.keys():
            for next_idx in row_multiplications[prev_idx]:
                merged_array[current_idx] = prev_factor[1][prev_idx] * next_factor[1][next_idx]
                current_idx += 1
        # Set up for the next two multiplications
        merged = (joined_vars, merged_array)
        prev_factor = merged

    # Sum over the variable to eliminate it
    resulting_variables = prev_factor[0]
    resulting_array = prev_factor[1]
    summed_array = np.zeros(len(resulting_array)//2)
    # Find the corresponding pairs of rows that must be summed together
    row_pairs = find_row_sum_pairs(var, resulting_variables)
    for i, row_pair in enumerate(row_pairs):
        summed_array[i] = resulting_array[row_pair[0]] + resulting_array[row_pair[1]]

    return [v for v in resulting_variables if v != var], summed_array


def relevant(parents: list[int], parents_in_evidence: list[int], evidence: dict[int,bool], row: int) -> bool:
    """Based on the parents' order for a given probability array, which parents are in the evidence, and whether those parents are true or false, determine if the row is consistent with the evidence based on our probability array construction convention

    Args:
        parents (list[int]): list of parents in order for some probability array
        parents_in_evidence (list[int]): list of parents which are in the evidence
        evidence (dict[int,bool]): records which variables in the evidence are true or false
        row (int): row index to determine if consistent with evidence

    Returns:
        bool: whether row number is consistent with evidence
    """
    for parent in parents_in_evidence:
        parent_idx = parents.index(parent)
        binary_posn = len(parents) - 1 - parent_idx
        switch_every = 2 ** binary_posn
        negative_row = ((row // switch_every) % 2) == 0
        if (negative_row and evidence[parent]) or ((not negative_row) and (not evidence[parent])):
            return False
    return True

def return_query_probabilities(queries: list[int], evidence: dict[int,bool], network: dict) -> np.array:
    """Given a Bayesian network and a list of query and evidence variables, return the probability distribution for all possible values of query variables

    Args:
        queries (list[int]): list of variables specified whose value probabilities we want to query
        evidence (list[tuple[int,bool]]): list of variables whose values are specified and hence affect query probabilities
        network (dict): underlying network which reveals probabilities of each node given its parents' values

    Returns:
        np.ndarray: probability distribution of possible combination values of each of the query variables (2^{#query variables}, 0 is all false and 2^{#query variables}-1 is all true)
    """
    # we need a map of NON-EVIDENCE variables to their respective factors (some of which will be shared) - factors are integers serving as references to numpy arrays
    factor_mappings = {int(i):[] for i in network.keys()}
    factors = {}
    # each factor has its own probability distribution - which may or may not depend on its parents
    for i, info in network.items():
        i = int(i)

        # if i is not in evidence, we have work to do
        if i in evidence.keys():
            continue

        parents = info["parents"]
        parents_in_evidence = [p for p in parents if p in evidence.keys()]
        parents_not_in_evidence = [p for p in parents if p not in evidence.keys()]
        # obtain the array of probabilities before considering evidence
        prob_array = np.array([pair[1] for pair in info["prob"]])

        # now filter the probability array so that it only contains entries consistent with the evidence
        relevant_rows = []
        for row in range(len(prob_array)):
            if relevant(parents, parents_in_evidence, evidence, row):
                relevant_rows.append(row)
        # quick sanity check
        assert len(relevant_rows) == 2 ** len(parents_not_in_evidence)

        # our base array size depends on the number of parents not in our evidence (and so take on both true and false values)
        new_prob_array = np.zeros(len(relevant_rows))
        # fill new_prob_array with relevant rows
        for i, row in enumerate(relevant_rows):
            new_prob_array[i] = prob_array[row]

        # double the size of our array so that it can contain the probabilities of 'i' being true and false
        final_prob_array = np.zeros(2 * len(new_prob_array))
        for i in range(len(prob_array)):
            final_prob_array[2*i] = 1 - new_prob_array[i]
            final_prob_array[2*i+1] = new_prob_array[i]

        # finally store into a tuple
        prob_tuple = ([parent for parent in parents] + [i], final_prob_array)
        # each involved variable needs THAT reference to prob_tuple
        factor_mappings[i].append(len(factors))
        for p in parents:
            if p not in evidence.keys():
                factor_mappings[p].append(len(factors))
        factors[len(factors)] = prob_tuple
    
    # figure out which variables need to be eliminated
    query_set = set(queries)
    evidence_set = set([pair[0] for pair in evidence.items()])
    hidden_vars = [i for i in range(len(network)) if i not in query_set and i not in evidence_set]
    # eliminate the hidden variables in an order based on their number of factors - more factors means eliminate sooner
    hidden_vars.sort(key=lambda i : -len(factor_mappings[i]))

    # now we go through and eliminate each variable
    for var in hidden_vars:
        resulting_vars, array = eliminate_variable(var, [factors[i] for i in factor_mappings[var]])
        for i in factor_mappings[var]:
            factors[i] = (resulting_vars, array)

    # now all factor keys for the dictionary should have the same reduced array of probability values
    result = factors[0][1]
    return result / np.sum(result) # for normalization

In [11]:
import json

queries = [0, 3]
evidence = {2:True}

with open('bn_test_2.json') as f:
    bayesian_network = json.load(f)
    print(return_query_probabilities(queries, evidence, bayesian_network))

IndexError: index 8 is out of bounds for axis 0 with size 8